In [ ]:
import sklearn
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
import shap
import pandas as pd
import sys
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing, linear_model
import re

sys.path.append('../LSP_Repo/')

import classifyFunctions
import helperFunctions as hf
import initFunctions as init

# Load my data
# Load Pickle
lightsheet_data = pd.read_pickle('lightsheet_data.pkl')

classifyDict = dict()
classifyDict['featureSel_scale'] = False # True, False
classifyDict['featureSel_linkage'] = 'average'  # 'average', 'complete', 'single'
classifyDict['featureSel_distance'] = 'correlation' # 'correlation, 'cosine', 'euclidean'
classifyDict['featureSel_filter'] = True
classifyDict['featureSel_agg'] = True
classifyDict['cluster_count'] = 10

classifyDict['data'] = 'cell_density' #cell_density, count
classifyDict['feature'] = 'abbreviation'
classifyDict['label'] = 'drug'

# classifyDict['model'] = 'LogRegElastic'
classifyDict['model'] = 'LogRegL2'

classifyDict['model_featureSel'] = 'L1'
classifyDict['model_classify'] = 'L2'

classifyDict['shuffle'] = True
classifyDict['includeSAL'] = True
classifyDict['include6FDET'] = True
classifyDict['remove_high_corr'] = True
classifyDict['corrThreshold'] = 0.9
classifyDict['gridCV'] = False


In [ ]:
# Set Paths to data and output
dirDict = dict()
rootDir = 'C:\OneDrive\MEng\KwanLab\cFosProject2\\'
dirDict['atlasDir'] = rootDir + 'Atlas\\'
dirDict['dataDir'] = rootDir + 'Data\\'
dirDict['B1'] =       dirDict['dataDir'] + 'lightSheetV1\\'
dirDict['B2'] =       dirDict['dataDir'] + 'lightSheetV2Rev\\'   #3/6/23 - Looking at the new, Realigned batch 2 data. #Realigned
dirDict['B2_Orig'] =  dirDict['dataDir'] + 'lightSheetV2\\'
dirDict['B3'] =       dirDict['dataDir'] + 'lightSheetV3\\'      #3/6/23 - Batch 3 with MDMA

batchSplit = False          # Splits drugs from the first batch of data, from the second, from the 3rd. Batch 1 is labeled with 'a' (aSAL, aKET, aPSI), Batch 3 (cKET, MDMA)
splitTag  = ['a', '', 'c']  # Appended the to beginning of data from the first batch (PSI, KET, SAL -> aPSI, KET, aSAL).
testSplit = False           # Splits an individual drug for the sake of examining self-similarity
oldBatch2 = False

debugOutputs = False        # Saves csvs at intervals
scalingFactor = True        # Applies 1/total_cells as a scaling factor per mouse.
debug_ROI = ['Dorsal nucleus raphe']
outputFormat = 'png'

switchDir = dict(testSplit=testSplit, batchSplit=batchSplit, splitTag=splitTag, oldBatch2=oldBatch2, debugOutputs=debugOutputs, scalingFactor=scalingFactor, debug_ROI=debug_ROI, outputFormat=outputFormat)

# Make directories, and add their strings to the directory dictionary.
dirDict = init.createDirs(rootDir, switchDir, dirDict)

In [ ]:
## See if I need to binarize, then try again.
myData = True

# Load data
if not myData:
    url = 'https://raw.githubusercontent.com/Sketchjar/MachineLearningHD/main/boston_data.csv'
    df = pd.read_csv(url); df.drop('Unnamed: 0',axis=1,inplace=True)
    X, y = df.drop('Target', axis=1), df.Target
else:
    
    if classifyDict['featureSel_filter']:
        pandasdf = hf.filter_and_agg_Data(lightsheet_data, classifyDict)

    if classifyDict['featureSel_agg']:
        ls_data_agg = hf.agg_cluster(pandasdf, classifyDict, dirDict)
    else:
        ls_data_agg = pandasdf.pivot(index='dataset', columns=classifyDict['feature'], values=classifyDict['data'])

    df = ls_data_agg.sample(frac=1)

    # X, y, featureNames, numYDict = hf.reformatData(pandasdf, classifyDict)

    # y = np.array([x[0:-1] for x in np.array(ls_data_agg.index)])
    yStr = np.array([x[0:-1] for x in np.array(df.index)])
    yDict = dict(zip(np.unique(yStr), range(1, len(np.unique(yStr))+1)))
    y = np.array([yDict[x] for x in yStr])

    X = df.reset_index(drop=True)
    
    featureNames = np.array(df.columns)
    # numYDict = {y: i for i, y in enumerate(np.unique(y))}
    numYDict = {value: key for key, value in yDict.items()}
    


In [ ]:
#Establish CV scheme
CV = KFold(n_splits=8, shuffle=True, random_state=10)

# Libraries for this section 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import shap
from sklearn import preprocessing

lb = preprocessing.LabelBinarizer()
lb.fit(y)
yAll = lb.transform(y)
CV_repeats = 100
splitCount = 8

for drug_idx in range(0,7):

    drugName = lb.classes_[drug_idx]
    print(f'Working on {drugName}')
    y = pd.Series(yAll[:,drug_idx])

    ix_training, ix_test = [], []
    # Loop through each fold and append the training & test indices to the empty lists above
    for fold in CV.split(df):
        ix_training.append(fold[0]), ix_test.append(fold[1])

    SHAP_values_per_fold = []

    np.random.seed(1) # Reproducibility 
    # Make a list of random integers between 0 and 10000 of length = CV_repeats to act as different data splits
    random_states = np.random.randint(10000, size=CV_repeats) 

    ######## Use a dict to track the SHAP values of each observation per CV repitition 

    shap_values_per_cv = dict()
    for sample in X.index:
        ## Create keys for each sample
        shap_values_per_cv[sample] = {} 
        ## Then, keys for each CV fold within each sample
        for CV_repeat in range(CV_repeats):
            shap_values_per_cv[sample][CV_repeat] = {}

    # Split data, establish model, fit model, make prediction, score model, print result
    for i, CV_repeat in enumerate(range(CV_repeats)): #-#-#
        #Verbose 
        # print('\n------------ CV Repeat number:', CV_repeat)
        #Establish CV scheme
        CV = KFold(n_splits=splitCount, shuffle=True, random_state=random_states[i]) # Set random state 

        ix_training, ix_test = [], []
        # Loop through each fold and append the training & test indices to the empty lists above
        for fold in CV.split(df):
            ix_training.append(fold[0]), ix_test.append(fold[1])
            
        ## Loop through each outer fold and extract SHAP values 
        for i, (train_outer_ix, test_outer_ix) in enumerate(zip(ix_training, ix_test)): 
            #Verbose
            # print('\n------ Fold Number:',i)
            X_train, X_test = X.iloc[train_outer_ix, :], X.iloc[test_outer_ix, :]
            y_train, y_test = y.iloc[train_outer_ix], y.iloc[test_outer_ix]

            if not myData:
                model = RandomForestRegressor(random_state=10) # Random state for reproducibility (same results every time)
                fit = model.fit(X_train, y_train)
                yhat = fit.predict(X_test)
                result = mean_squared_error(y_test, yhat)

                # Use SHAP to explain predictions
                explainer = shap.Explainer(model)
                shap_values = explainer.shap_values(X_train)
            else:
                model = sklearn.linear_model.LogisticRegression(penalty='l2', solver='liblinear', max_iter=1000, dual=True)
                pipeline = make_pipeline(preprocessing.RobustScaler(), model)
                pipelineT = make_pipeline(preprocessing.RobustScaler())
                pipelineT.fit(X_train, y_train)
                
                # Transform data while preserving pandas dataframe structure and index
                X_train_trans = pd.DataFrame(pipelineT.transform(X_train), columns=X_train.columns)
                X_train_trans.set_index(X_train.index)
                # X_train_trans = X_train.copy()

                fit = pipeline.fit(X_train, y_train)

                explainer = shap.Explainer(pipeline._final_estimator, X_train_trans, feature_names=X.columns, max_iter=1000)
                shap_values = explainer.shap_values(X_train_trans)
                # shap.plots.beeswarm(shap_values)
                
            # Extract SHAP information per fold per sample 
            for i, test_index in enumerate(test_outer_ix):
                shap_values_per_cv[test_index][CV_repeat] = shap_values[i] #-#-#

    # Establish lists to keep average Shap values, their Stds, and their min and max
    average_shap_values, stds, ranges = [],[],[]

    for i in range(0,len(df)):
        df_per_obs = pd.DataFrame.from_dict(shap_values_per_cv[i]) # Get all SHAP values for sample number i
        # Get relevant statistics for every sample 
        average_shap_values.append(df_per_obs.mean(axis=1).values) 
        stds.append(df_per_obs.std(axis=1).values)
        ranges.append(df_per_obs.max(axis=1).values-df_per_obs.min(axis=1).values)

    new_index = [ix for ix_test_fold in ix_test for ix in ix_test_fold]
    
    shap.summary_plot(np.array(average_shap_values), X, show = False)
    plt.title(f'{drugName} - Average SHAP values after {CV_repeats}x cross-validation')
    plt.show()
